In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import re
import pandas as pd

#Recipe_NLG = pd.read_csv("Dataset_Recipe NLG\Recipe NLG_archive\RecipeNLG_dataset.csv", sep = ",")
Recipe_foodcom = pd.read_csv("drive/My Drive/DataScientest - NLP_Mail/Foodcom Recipes _ archive/RAW_recipes.csv", sep = ",")

Recipe_foodcom.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [11]:
# enlever les colones non-essentiels
Recipe_foodcom2 = Recipe_foodcom.drop(['id', 'minutes', 'contributor_id', 'submitted', 'nutrition', 'description', 'tags', 'n_ingredients'], axis = 1)
Recipe_foodcom2.head()

,name,n_steps,steps,ingredients
0,arriba baked winter squash mexican style,11,"['make a choice and proceed with recipe', 'dep...","['winter squash', 'mexican seasoning', 'mixed ..."
1,a bit different breakfast pizza,9,"['preheat oven to 425 degrees f', 'press dough...","['prepared pizza crust', 'sausage patty', 'egg..."
2,all in the kitchen chili,6,"['brown ground beef in large pot', 'add choppe...","['ground beef', 'yellow onions', 'diced tomato..."
3,alouette potatoes,11,['place potatoes in a large pot of lightly sal...,"['spreadable cheese with garlic and herbs', 'n..."
4,amish tomato ketchup for canning,5,['mix all ingredients& boil for 2 1 / 2 hours ...,"['tomato juice', 'apple cider vinegar', 'sugar..."


In [12]:
def pre_process_recipe(recette):
    # pre-process "Steps" column: enlever les signes puis splitter le colone en list d'étapes 
    recette = recette.lower().split("', '")   # spliter
    filtered = []
    for step in recette:
        step = re.sub(r'[\'\,\]\[\-\&\"\/\!\^\#]', ' ', step)     # remplacer les signes par les espaces
        step = re.sub(r'[\s]+', ' ', step)    # remplacer les espaces consécutives en un seul espace
        filtered.append(step)
    #regex2 = r2.findall(text_recette)
    return filtered

def pre_process_ingrd(ingredient_string):
    # pre-process ingredients : remove special characters, double spaces & punctuations
    ingredient_string.lower()
    ingredient_string = re.sub(r'[\'\,\]\[\-\&\"\/\!\^\#]', ' ', ingredient_string)
    ingredient_string = re.sub(r'[\s]+', ' ', ingredient_string)
    ingredient_string = re.sub(r' and', '', ingredient_string)
    ingredient_string = '[ingredients]'+ ingredient_string +'[/ingredients]' # add word ingredient at the beginning of ingredient text
    return ingredient_string

print(Recipe_foodcom2['ingredients'].tolist()[3])
print(pre_process_ingrd(Recipe_foodcom['ingredients'].tolist()[3]))

['spreadable cheese with garlic and herbs', 'new potatoes', 'shallots', 'parsley', 'tarragon', 'olive oil', 'red wine vinegar', 'salt', 'pepper', 'red bell pepper', 'yellow bell pepper']
[ingredients] spreadable cheese with garlic herbs new potatoes shallots parsley tarragon olive oil red wine vinegar salt pepper red bell pepper yellow bell pepper [/ingredients]


In [13]:
def Insert(list, element):
    # function to insert ingredient at the beginning of list of steps, considered as elemnt #0 in the list
    list.insert(0, element)
    return list

In [14]:
# Apply above functions to the "step" column
Recipe_foodcom_p = Recipe_foodcom2
Recipe_foodcom_p['steps'] = Recipe_foodcom_p['steps'].apply(lambda steps : pre_process_recipe(steps))  # split recipe into list of steps
Recipe_foodcom_p['ingredients'] = Recipe_foodcom_p['ingredients'].apply(lambda ingredients : pre_process_ingrd(ingredients)) 
Recipe_foodcom_p['steps']= Recipe_foodcom_p.apply(lambda row: Insert(row['steps'], row['ingredients']), axis = 1)

# check one recipe 
for step in Recipe_foodcom_p['steps'].tolist()[3]:
    print(step)

[ingredients] spreadable cheese with garlic herbs new potatoes shallots parsley tarragon olive oil red wine vinegar salt pepper red bell pepper yellow bell pepper [/ingredients]
 place potatoes in a large pot of lightly salted water and bring to a gentle boil
cook until potatoes are just tender
drain
place potatoes in a large bowl and add all ingredients except the alouette 
mix well and transfer to a buttered 8x8 inch glass baking dish with 2 inch sides
press the potatoes with a spatula to make top as flat as possible
set aside for 2 hours at room temperature
preheat oven to 350 f
spread alouette evenly over potatoes and bake 15 minutes
divide between plates
garnish with finely diced red and yellow bell peppers 


In [15]:
# check one recipe 
for step in Recipe_foodcom_p['steps'].tolist()[5]:
    print(step)

[ingredients] milk vanilla ice cream frozen apple juice concentrate apple [/ingredients]
 combine ingredients in blender
cover and blend until smooth
sprinkle with ground cinnamon
makes about 2 cups 


In [19]:
# Add column calculating the number of steps
Recipe_foodcom_p['nb_steps'] = Recipe_foodcom_p.apply(lambda row : len(row['steps']), axis = 1) 
# filter out recipes with length nb_steps <= 4
Recipe_foodcom_p2 = Recipe_foodcom_p[Recipe_foodcom_p['nb_steps']>4]
Recipe_foodcom_p2.shape

# Garder seulement 10 000 lignes pour réduire temps de calcul
data_size = 200000
Recipe_foodcom_red = Recipe_foodcom_p2.iloc[0:data_size]

# Create input = 4 premiers steps
# Output = prochain étape
Recipe_foodcom_red['Input_4'] = Recipe_foodcom_red.apply(lambda row : ' '.join(row['steps'][:4]), axis = 1) 
Recipe_foodcom_red['Target_4'] = Recipe_foodcom_red.apply(lambda row : row['steps'][4], axis = 1) 
Recipe_foodcom_red = Recipe_foodcom_red.drop(['n_steps', 'steps', 'ingredients'], axis = 1)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [20]:
# Ajouter "[start]" et "end" aux deux bouts du text 
Recipe_foodcom_red['Target_4'] = Recipe_foodcom_red.apply(lambda row : "[start] "+ row['Target_4'] + " [end]", axis = 1) 
Recipe_foodcom_red.head()

,name,nb_steps,Input_4,Target_4
0,arriba baked winter squash mexican style,12,[ingredients] winter squash mexican seasoning ...,[start] for spicy squash drizzle olive oil or ...
1,a bit different breakfast pizza,10,[ingredients] prepared pizza crust sausage pat...,[start] cut sausage into small pieces [end]
2,all in the kitchen chili,7,[ingredients] ground beef yellow onions diced ...,[start] add kidney beans if you like beans in ...
3,alouette potatoes,12,[ingredients] spreadable cheese with garlic he...,[start] place potatoes in a large bowl and add...
5,apple a day milk shake,5,[ingredients] milk vanilla ice cream frozen ap...,[start] makes about 2 cups [end]


In [21]:
# before tokenisation, Saving data frame for reference
Recipe_foodcom_red.to_csv(r'drive/My Drive/DataScientest - NLP_Mail/Model3_Encoder_Decoder/Recipe_foodcom_red.csv') 

===== ====== End preprocess text ==================

======================TOKENNISATION & PREPARATION DU DATASET ===============================

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

max_input_len = 70 # fixer à 70 la longeur max des étapes d'entrées
max_target_len = 20  # fixer à 18 la longeur max de l'étape à prédire 
size_vocab = 70000

# Initier puis adapter les vectorizers pour Input text & Target Text
input_vectorizer= TextVectorization( max_tokens=size_vocab, output_mode='int',  output_sequence_length=max_input_len)
target_vectorizer = TextVectorization(max_tokens=size_vocab, output_mode='int', output_sequence_length=max_target_len)

input_vectorizer.adapt(Recipe_foodcom_red['Input_4'].tolist()+ Recipe_foodcom_red['Target_4'].tolist())
target_vectorizer.adapt(Recipe_foodcom_red['Input_4'].tolist()+ Recipe_foodcom_red['Target_4'].tolist())

In [ ]:
# function pour reformater
import tensorflow as tf

def format_dataset(input, target):
    encoded_input = input_vectorizer(input)   # Note: vectorizer can only process list of strings, not single string
    encoded_target = target_vectorizer(target)
    return ({ 
        "encoder_input": encoded_input,
        "decoder_input": encoded_target[:, :-1]}
        , encoded_target[:, 1:])

def create_dataset (dataframe):
# Charger les données dans object dataset 
  dataset_text = tf.data.Dataset.from_tensor_slices((dataframe['Input_4'].tolist(), dataframe['Target_4'].tolist()))
# puis appliquer fonction d'encodage pour obtenir formater les données matching la configuration requis pour 
  return dataset_text.batch(32).map(format_dataset)


training_portion = 0.8
data_size = 10000

Recipe_foodcom_train = Recipe_foodcom_red.iloc[0: 180000]
Recipe_foodcom_val = Recipe_foodcom_red.iloc[180000:]

dataset_training = create_dataset(Recipe_foodcom_train)
dataset_validation = create_dataset(Recipe_foodcom_val)

In [ ]:
# Vérifier contenu du dataset
for input, target in dataset_training.take(count=1):
    print(input)
    print(target)

===================== CONSTRUCTION DU MODEL & TRAINING =================================================

In [ ]:
from tensorflow.keras import layers
import tensorflow.keras as keras

embed_dim = 256
unit_gru = 1024
size_vocab = 200000

# DEFINE ENCODER
encoder_input = keras.Input(shape = (None,), dtype = "int64", name = "encoder_input")
x = layers.Embedding(size_vocab, embed_dim, mask_zero = True)(encoder_input)
encoder_output = layers.GRU(units = unit_gru)(x)

# DEFINE DECODER
decoder_input = keras.Input(shape = (None,), dtype = "int64", name = "decoder_input")
x = layers.Embedding(size_vocab, embed_dim, mask_zero = True)(decoder_input)
decoder_GRU = layers.GRU(units = unit_gru, return_sequences = True)
x = decoder_GRU(x, initial_state = encoder_output)
x = layers.Dropout(0.5)(x)
decoder_output = layers.Dense(size_vocab+1, activation = "softmax")(x)

total_model = keras.Model([encoder_input, decoder_input], decoder_output)

In [ ]:
total_model.compile(optimizer = "rmsprop", loss = "sparse_categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
total_model.fit(dataset_training, epochs = 10, validation_data = dataset_validation )

Epoch 1/10
5625/5625 [==============================] - 3072s 545ms/step - loss: 2.2897 - accuracy: 0.3629 - val_loss: 2.0806 - val_accuracy: 0.4054
Epoch 2/10
5625/5625 [==============================] - 2982s 530ms/step - loss: 2.0545 - accuracy: 0.4097 - val_loss: 1.9666 - val_accuracy: 0.4230
Epoch 3/10
5625/5625 [==============================] - 3007s 535ms/step - loss: 1.9562 - accuracy: 0.4217 - val_loss: 1.9511 - val_accuracy: 0.4294
Epoch 4/10
5625/5625 [==============================] - 2973s 529ms/step - loss: 1.9427 - accuracy: 0.4267 - val_loss: 1.9629 - val_accuracy: 0.4316
Epoch 5/10
5625/5625 [==============================] - 2976s 529ms/step - loss: 1.9430 - accuracy: 0.4300 - val_loss: 1.9664 - val_accuracy: 0.4329
Epoch 6/10
5625/5625 [==============================] - 3009s 535ms/step - loss: 1.9445 - accuracy: 0.4311 - val_loss: 1.9709 - val_accuracy: 0.4332
Epoch 7/10
5625/5625 [==============================] - 2943s 523ms/step - loss: 1.9492 - accuracy: 0.4304

================= SAUVEGARDE MODEL =======================

In [ ]:
total_model.save("drive/MyDrive/DataScientest - NLP_Mail/Model3_Encoder_Decoder/Model_10_03")

In [ ]:
# Load model for testing
#model2 = keras.models.load_model("drive/MyDrive/DataScientest - NLP_Mail/Model3_Encoder_Decoder/Model_10_03")

===================== FONCTION DE GENERATION DE TEXT ==========================

In [ ]:
# Prepare fonction générateur de text
import numpy as np
voca_index = target_vectorizer.get_vocabulary()
voca_dic = dict(zip(range(len(voca_index)), voca_index ))
max_length = 18

def decode_sequence(input):
    input_encoded = input_vectorizer([input])
    target = "[start]"
    for i in range(max_length):
       target_encoded = target_vectorizer([target])
       next_prediction = total_model.predict([input_encoded, target_encoded])
       token_index = np.argmax( next_prediction[0, i, :])
       sampled_mot = voca_dic[token_index]
       target += " " + sampled_mot
  #     if sampled_mot == "[end]":
   #      break
    return target
  

In [ ]:
test_input = "[ingredients] milk vanilla ice cream frozen apple juice concentrate apple combine ingredients in blender cover and blend until smooth sprinkle with ground cinnamon"
target = decode_sequence(test_input)

print(target)

[start] add 1 cup of the sugar and stir to combine end  end end end end end end


In [ ]:
test2 = "[ingredients] spreadable cheese with garlic herbs new potatoes shallots parsley tarragon olive oil red wine vinegar salt pepper red bell pepper yellow bell pepper place potatoes in a large pot of lightly salted water and bring to a gentle boil cook until potatoes are just tender drain place potatoes in a large bowl and add all ingredients except the alouette" 
target = decode_sequence(test2)
print(target)

NameError: ignored